<a href="https://colab.research.google.com/github/FaisalWant/pandas_dataPipeline/blob/main/pandas_DataPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [3]:
import boto3 
import pandas as pd
from io import StringIO



In [ ]:
s3= boto3.resource('s3') 
bucket= s3.Bucket('deutsche-boerse-xetra-pds') 


In [ ]:
bucket_obj= bucket.objects.filter(Prefix='2021-03-15') 
objects= [obj for obj in bucket_obj]

In [ ]:
for obj in objects:  
  csv_obj= bucket.Object(key='2021-03-15/2021-03-15_BINS_XETR15.csv').get().get('Body').read().decode('utf-8')
  data= StringIO(csv_obj) 
  df= pd.read_csv(data, delimiter=",") 
  df_all = df_all.append(df, ignore_index=True)  

In [ ]:
data = StringIO(csv_obj) 

df= pd.read_csv(data, delimiter=',') 

In [2]:
import pandas as pd
tech_url= "https://andybek.com/pandas-tech"
tech= pd.read_csv(tech_url) 

In [ ]:
tech